In [26]:
from ipynb.fs.full.common import *

Variables

In [27]:
ACTIVE_SHAM = 'Active'
SAMPLE = 'Preeti'
RANGE = 1

Importing all packages

In [28]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy.fft import fft
import time
import mne
import matplotlib
import matplotlib.pyplot as plt
import os
from scipy.stats import kurtosis, zscore, ttest_ind
from mne.preprocessing import create_ecg_epochs, create_eog_epochs, read_ica
from mne.time_frequency import tfr_morlet, tfr_array_morlet, morlet, AverageTFR
from itertools import product
import pywt
from scipy.signal import hilbert
from mne_connectivity import spectral_connectivity_epochs
from mne_connectivity.viz import plot_sensors_connectivity
import networkx as nx
import numpy as np 
import re
from scipy.cluster.hierarchy import dendrogram, linkage
from colabcode import ColabCode
from nilearn import plotting
from nilearn.connectome import ConnectivityMeasure
from nilearn.maskers import MultiNiftiLabelsMasker
from nilearn import datasets
from communities.algorithms import hierarchical_clustering

matplotlib.use('TkAgg')

## LOAD DATA

In [29]:
# 2Pre-TDCS : .easy, # 3TDCS : .easy, # 4Post-TDCS : .easy
import glob
folder_path = os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', '32electrodes', ACTIVE_SHAM)
# folders = ['1Pre-BML', '2Pre-TDCS', '3TDCS', '4Post-TDCS', '5Post-BML']
folders = ['2Pre-TDCS', '3TDCS', '4Post-TDCS']
all_files = []
for folder in folders:
    ext = '.eeg' if folder[0] in ['1', '5'] else 'Close.easy'
    for _ in glob.glob(os.path.join(folder_path, folder, SAMPLE) + '/*' + ext):
        all_files.append(_)
print(len(all_files), all_files)
GROUP1 = RANGE
raw_1 = data_transformation_easy(all_files[RANGE])
raw_1 = g1_preprocess(raw_1)
GROUP2 = RANGE+1
raw_2 = data_transformation_easy(all_files[RANGE+1])
raw_2 = g1_preprocess(raw_2)

48 ['/home/vishwani/Downloads/IITD/Depression-IITD/Depression-Sample-dataset-AIIMS/32electrodes/Active/2Pre-TDCS/Preeti/20230718201550_Preeti singh_22.08.23-01_Eye Close.easy', '/home/vishwani/Downloads/IITD/Depression-IITD/Depression-Sample-dataset-AIIMS/32electrodes/Active/3TDCS/Preeti/20230718210007_Preeti singh_22.08.23_01_Rest_Eye Close.easy', '/home/vishwani/Downloads/IITD/Depression-IITD/Depression-Sample-dataset-AIIMS/32electrodes/Active/3TDCS/Preeti/20230826191815_PreetiSingh_06.09.23_19_Rest_Eye Close.easy', '/home/vishwani/Downloads/IITD/Depression-IITD/Depression-Sample-dataset-AIIMS/32electrodes/Active/3TDCS/Preeti/20230826220508_PreetiSingh_08.09.23_20_Rest_Eye Close.easy', '/home/vishwani/Downloads/IITD/Depression-IITD/Depression-Sample-dataset-AIIMS/32electrodes/Active/3TDCS/Preeti/20230826155056_PreetiSingh_06.09.23_10_18A_Rest_Eye Close.easy', '/home/vishwani/Downloads/IITD/Depression-IITD/Depression-Sample-dataset-AIIMS/32electrodes/Active/3TDCS/Preeti/20230824165000

/home/vishwani/Downloads/IITD/Depression-IITD/common.ipynb:251: RuntimeWarning: filter_length (155001) is longer than the signal (50001), distortion is likely. Reduce filter length or filter a longer signal.
  ]


EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Creating RawArray with float64 data, n_channels=32, n_times=149999
    Range : 0 ... 149998 =      0.000 ...   299.996 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.01 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, non-linear phase, causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hann window with 0.0546 passband ripple and 44 dB stopband attenuation
- Lower transition bandwidth: 0.01 Hz
- Upper transition bandwidth: 11.25 Hz
- Filter length: 155001 samples (310.002 s)



/home/vishwani/Downloads/IITD/Depression-IITD/common.ipynb:251: RuntimeWarning: filter_length (155001) is longer than the signal (50001), distortion is likely. Reduce filter length or filter a longer signal.
  ]


EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


SSP projectors applied...


### Epoching

In [30]:
duration = 4.0
overlap = 2.0 

samples_per_epoch = int(duration * SFREQ)
samples_per_overlap = int(overlap * SFREQ)

# Manually created events
start, stop = 0, samples_per_epoch
events = []
while stop <= len(raw_1):
    events.append([start, 0, 1]) 
    start += samples_per_overlap
    stop += samples_per_overlap

events = np.array(events)

3.1 Segmenting epochs for Group 1

In [31]:
epochs_1 = mne.Epochs(raw_1, events, tmin=0, tmax=duration, baseline=None, detrend=1,
                    picks=None, preload=True, reject=None)
# raw_1.plot(n_channels=len(raw_1.info['ch_names']), events=events, event_color={1:'r'}, scalings=SCALINGS)
# epochs_1.plot(n_channels=len(raw_1.info['ch_names']), event_color={1:'r'}, events=events, scalings=SCALINGS)

Not setting metadata
49 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
15 bad epochs dropped


3.2 Segmenting epochs for Group 2

In [32]:
epochs_2 = mne.Epochs(raw_2, events, tmin=0, tmax=duration, baseline=None, detrend=1,
                    picks=None, preload=True, reject=None)
# raw_2.plot(n_channels=len(raw_1.info['ch_names']), events=events, event_color={1:'r'}, scalings=SCALINGS)
# epochs_2.plot(n_channels=len(raw_1.info['ch_names']), event_color={1:'r'}, events=events, scalings=SCALINGS)

Not setting metadata
49 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
15 bad epochs dropped


### 1. PSD graph plot between MDD and Control group

For resting state EEG channel, analyzing the overall average power across channels provides a holistic view of the brain's activity without emphasizing the specificity of individual channels.

In [33]:
# Amplitude plots between both groups
epochs_1_avg = epochs_1['1'].average()
epochs_2_avg = epochs_2['1'].average()
# evokeds = dict(epochs_1=epochs_1)
fig_MDD = epochs_1_avg.plot(titles=GROUP1, time_unit = 'ms')
fig_control = epochs_2_avg.plot(titles=GROUP2, time_unit = 'ms')

1.1 PSD using plot_psd

In [61]:
fig, ax = plt.subplots()
#plot_psd uses welch method for continuous data
raw_1.plot_psd(picks=raw_1.info['ch_names'], average=True, fmin=FMIN, fmax=FMAX, ax=ax, show=False, color='blue', dB=False)
raw_2.plot_psd(picks=raw_2.info['ch_names'], average=True, fmin=FMIN, fmax=FMAX, ax=ax, show=False, color='red', dB=False)

# dB = True plots PSD in decibels (logarithmic)
# different n_fft compared to psd_array_welch
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Power/Frequency (microV^2/Hz)')
ax.set_title('Power Spectral Density Comparison')

ax.text(0.8, 0.9, GROUP1, color='blue', transform=ax.transAxes)
ax.text(0.8, 0.85, GROUP2, color='red', transform=ax.transAxes)

plt.ylim(0, 8)
plt.show()

fig.savefig(f'comparison-results/PSD - {GROUP1} vs {GROUP2}.png')


NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 4.096 (s)
NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 4.096 (s)


1.3 Statistical testing between psds of raw

1.3.1 Plotting Histogram

Shows if the data is bell shaped, skewed (positive or negative), uniform etc. Central tendency - highest peak in histogram. Spread - width of distribution

In [35]:
# g1_avg = np.mean(raw_1.get_data(), axis=0)
# plt.hist(g1_avg, bins='auto', alpha=0.7, edgecolor='black')
# plt.title(f'Histogram of {GROUP1} Data - averaged across all channels')
# plt.xlabel('Values')
# plt.ylabel('Frequency')
# plt.show()

In [36]:
# g2_avg = np.mean(raw_2.get_data(), axis=0)
# plt.hist(g2_avg, bins='auto', alpha=0.7, edgecolor='black')
# plt.title(f'Histogram of {GROUP2} Data - averaged across all channels')
# plt.xlabel('Values')
# plt.ylabel('Frequency')
# plt.show()

In [37]:
# print(f"{GROUP1} mean: ", np.mean(g1_avg, axis=0))
# print(f"{GROUP2} mean: ", np.mean(g2_avg, axis=0))

1.3.2 Calculate variance of both groups

In [38]:
# variance_1 = np.var(psd_1, ddof=1) 
# variance_2 = np.var(psd_2, ddof=1)
# print(f"{GROUP1} variance: ", variance_1)
# print(f"{GROUP2} variance: ", variance_2)

1.3.3 Find differences between the means of two independent groups

For example, null hypothesis: there is no significant difference between groups (ie means are equal).
alternative hypothesis: there is a significant difference between groups
t-test : compares the differences between group means to expected variability in data

In [39]:
# p_values = np.zeros(32)
# t = np.zeros(32)
# for i in range(32):
#     t[i], p_values[i] = ttest_ind(psd_1[i, :], psd_2[i, :])
# # independent t-test comparison : equal_var = True
# # welch's t-test comparison: equal_var = False
# print("Raw Data PSD Comparison Results:")
# print("T-statistic:", t)
# print("P-values:", p_values) # probability of getting t-statistic >= what we got

In [40]:
# t_statistic, p_values = ttest_ind(np.mean(psd_1, axis=0), np.mean(psd_2, axis=0), equal_var=False)
# # independent t-test comparison : equal_var = True
# # welch's t-test comparison: equal_var = False
# print("Raw Data PSD Comparison Results:")
# print("T-statistic:", t_statistic)
# print("P-values:", p_values) # probability of getting t-statistic >= what we got

If p value < 0.005, we reject the null hypothesis

### 2. PLI and construction of brain function matrices

In [41]:
freq_bands = [(0.01, 4), (4,8), (8, 10), (10, 13), (13, 15), (15, 22), (22, 30), (30,45)]
mapping = {0: 'delta', 1: 'theta', 2: 'alpha1', 3:'alpha2', 4:'beta1', 5:'beta2', 6: 'beta3', 7: 'gamma'}
connectivity = {}

In [42]:
# pli method always gives positive correlations
def connectivity_matrix(epochs, i):
    return spectral_connectivity_epochs(
    epochs, method='pli', mode='multitaper', sfreq=SFREQ,
    fmin=freq_bands[i][0], fmax=freq_bands[i][1], faverage=True, n_jobs=1)


In [43]:
n_channels = len(raw_1.info['ch_names'])
for i in range(len(freq_bands)):
    con = connectivity_matrix(epochs_1, i)
    connectivity[f'{GROUP1}-{mapping[i]}'] = con.get_data().reshape((n_channels, n_channels))

    con = connectivity_matrix(epochs_2, i)
    connectivity[f'{GROUP2}-{mapping[i]}'] = con.get_data().reshape((n_channels, n_channels))

    # Make matrix symmetric
    for row in range(n_channels):
        for col in range(n_channels):
            connectivity[f'{GROUP1}-{mapping[i]}'][row][col] = connectivity[f'{GROUP1}-{mapping[i]}'][col][row]
            connectivity[f'{GROUP2}-{mapping[i]}'][row][col] = connectivity[f'{GROUP2}-{mapping[i]}'][col][row]
    

Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 0.2Hz..4.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing connectivity for epo

/tmp/ipykernel_19466/1345682627.py:3: RuntimeWarning: fmin=0.010 Hz corresponds to 0.040 < 5 cycles based on the epoch length 4.002 sec, need at least 500.000 sec epochs or fmin=1.249. Spectrum estimate will be unreliable.
  return spectral_connectivity_epochs(


    computing connectivity for epoch 22
    computing connectivity for epoch 23
    computing connectivity for epoch 24
    computing connectivity for epoch 25
    computing connectivity for epoch 26
    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    computing connectivity for epoch 30
    computing connectivity for epoch 31
    computing connectivity for epoch 32
    computing connectivity for epoch 33
    computing connectivity for epoch 34
    assembling connectivity matrix
[Connectivity computation done]
Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 0.2Hz..4.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PL

/tmp/ipykernel_19466/1345682627.py:3: RuntimeWarning: fmin=0.010 Hz corresponds to 0.040 < 5 cycles based on the epoch length 4.002 sec, need at least 500.000 sec epochs or fmin=1.249. Spectrum estimate will be unreliable.
  return spectral_connectivity_epochs(


    computing connectivity for epoch 18
    computing connectivity for epoch 19
    computing connectivity for epoch 20
    computing connectivity for epoch 21
    computing connectivity for epoch 22
    computing connectivity for epoch 23
    computing connectivity for epoch 24
    computing connectivity for epoch 25
    computing connectivity for epoch 26
    computing connectivity for epoch 27
    computing connectivity for epoch 28
    computing connectivity for epoch 29
    computing connectivity for epoch 30
    computing connectivity for epoch 31
    computing connectivity for epoch 32
    computing connectivity for epoch 33
    computing connectivity for epoch 34
    assembling connectivity matrix
[Connectivity computation done]
Replacing existing metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (1

In [44]:
print((connectivity.keys()))

dict_keys(['1-delta', '2-delta', '1-theta', '2-theta', '1-alpha1', '2-alpha1', '1-alpha2', '2-alpha2', '1-beta1', '2-beta1', '1-beta2', '2-beta2', '1-beta3', '2-beta3', '1-gamma', '2-gamma'])


### 3. Thresholding - M1 -  Small World Index

Preserves small-world properties in both groups to ensure that any observed differences in connectivity are not biased by the thresholding method.
Preserving these properties ensures that information can be transmitted quickly and effectively across the network.

In [151]:
columns=[]

In [152]:
df = pd.read_excel("comparison-results/output.xlsx")
main_column_names = ['swi', 'links']
subcols = ('delta-g1', 'delta-g2', 'theta-g1', 'theta-g2', 'alpha1-g1', 'alpha1-g2', 'alpha2-g1', 'alpha2-g2', 
           'beta1-g1', 'beta1-g2', 'beta2-g1', 'beta2-g2', 'beta3-g1', 'beta3-g2', 'gamma-g1', 'gamma-g2')

# Define subcolumn names for each main column
subcolumn_names = {
    'swi': subcols,
    'links': subcols,
}

# Create a list of tuples representing column names
columns_tuples = [(main_col, sub_col) for main_col in main_column_names for sub_col in subcolumn_names[main_col]]

# Create a MultiIndex for the columns
columns = pd.MultiIndex.from_tuples(columns_tuples)


# # Assign the MultiIndex columns to the DataFrame
# df.columns = columns
# df = pd.DataFrame(columns=columns)

# # Display the DataFrame
# print("DataFrame with columns and sub-columns:")
# df.head()

In [154]:
optimal_threshold = {}
edges_1 = {}
edges_2 = {}
# Generating 50 random networks with same number of vertices and edges 
random_networks = [nx.gnm_random_graph(n_channels, 496) for _ in range(50)]
thresholds = np.arange(0.1, 0.9, 0.002)

for i in mapping:
    edges_1[mapping[i]] = []
    edges_2[mapping[i]] = []
    ratios = []
    for threshold in thresholds:
        # TODO: Cross check should we ignore thresholds which result in matrix to be not connected ?
        g1 = calculate_avergae_components(nx.from_numpy_array(connectivity[f'{GROUP1}-{mapping[i]}'] > threshold))
        Lw_1_binarized, CC_1_binarized = g1[0], g1[3]

        g2 = calculate_avergae_components(nx.from_numpy_array(connectivity[f'{GROUP2}-{mapping[i]}'] > threshold))
        Lw_2_binarized, CC_2_binarized = g2[0], g2[3]

        swi_ratio_binarized =  (CC_1_binarized*Lw_2_binarized)/(Lw_1_binarized*CC_2_binarized) if Lw_1_binarized and CC_2_binarized else 0
        ratios.append(swi_ratio_binarized)
        
    optimal_threshold[mapping[i]] = thresholds[np.argmax(np.abs(ratios))] #TODO: ratios - swi_ratio ? OR remove swi_ratio and related code above
    t_stat, p_value = ttest_ind(np.mean(connectivity[f'{GROUP1}-{mapping[i]}'], axis=0), np.mean(connectivity[f'{GROUP2}-{mapping[i]}'], axis=0)) 
    if p_value < 0.005: # Reject the null hypotheses
        Crand = np.mean([calculate_avergae_components(nx.from_numpy_array((nx.to_numpy_array(G) > optimal_threshold[mapping[i]])))[3] for G in random_networks])
        Lrand = np.mean([calculate_avergae_components(nx.from_numpy_array((nx.to_numpy_array(G) > optimal_threshold[mapping[i]])))[0] for G in random_networks])

        graph1 = calculate_avergae_components(nx.from_numpy_array(connectivity[f'{GROUP1}-{mapping[i]}'] > optimal_threshold[mapping[i]]))
        Lw_1_binarized, CC_1_binarized = graph1[0], graph1[3]

        graph2 = calculate_avergae_components(nx.from_numpy_array(connectivity[f'{GROUP2}-{mapping[i]}'] > optimal_threshold[mapping[i]]))
        Lw_2_binarized, CC_2_binarized = graph2[0], graph2[3]

        swi_binarized_1 =  (CC_1_binarized/Crand)/(Lw_1_binarized/Lrand)
        swi_binarized_2 = (CC_2_binarized/Crand)/(Lw_2_binarized/Lrand)
        print(swi_binarized_1, "Cc1 ", CC_1_binarized, "Lw1 ", Lw_1_binarized)
        print(swi_binarized_2, "Cc2 ", CC_2_binarized, "Lw2 ", Lw_2_binarized)  

        edges_1[mapping[i]].append(nx.from_numpy_array(connectivity[f'{GROUP1}-{mapping[i]}'] > optimal_threshold[mapping[i]]).number_of_edges())
        edges_2[mapping[i]].append(nx.from_numpy_array(connectivity[f'{GROUP2}-{mapping[i]}'] > optimal_threshold[mapping[i]]).number_of_edges())

        print(f"{mapping[i]} - Optimal Threshold: {optimal_threshold[mapping[i]]} (Significant)", p_value)
        df.at[GROUP1, ('swi', f'{mapping[i]}-g1')] = swi_binarized_1
        df.at[GROUP1, ('swi', f'{mapping[i]}-g2')] = swi_binarized_2
    else:
        df.at[GROUP1, ('swi', f'{mapping[i]}-g1')] = None
        df.at[GROUP1, ('swi', f'{mapping[i]}-g2')] = None

In [ ]:
df.to_excel("comparison-results/output.xlsx", index=False)

### 4. Binarization of functional connectivity

In [ ]:
binarized_matrix = {}
for i in mapping:
    binarized_matrix[f'{GROUP1}-{mapping[i]}'] = connectivity[f'{GROUP1}-{mapping[i]}'] > optimal_threshold[mapping[i]]
    binarized_matrix[f'{GROUP2}-{mapping[i]}'] = connectivity[f'{GROUP2}-{mapping[i]}'] > optimal_threshold[mapping[i]]
print(binarized_matrix.keys())

dict_keys(['1-delta', '2-delta', '1-theta', '2-theta', '1-alpha1', '2-alpha1', '1-alpha2', '2-alpha2', '1-beta1', '2-beta1', '1-beta2', '2-beta2', '1-beta3', '2-beta3', '1-gamma', '2-gamma'])


Find number of links in each graph

In [ ]:
for i in mapping:
    links_1 = 0
    links_2 = 0

    for row in range(n_channels):
        for col in range(row+1):
            if binarized_matrix[f'{GROUP1}-{mapping[i]}'][row][col] == 1:
                links_1+=1
            if binarized_matrix[f'{GROUP2}-{mapping[i]}'][row][col] == 1:
                links_2+=1

    print(f'{GROUP1}-{mapping[i]}-number of links: K=', links_1)
    print(f'{GROUP2}-{mapping[i]}-number of links: K=', links_2)

    df.at[GROUP1, ('links', f'{mapping[i]}-g1')] = links_1
    df.at[GROUP1, ('links', f'{mapping[i]}-g2')] = links_2

1-delta-number of links: K= 21
2-delta-number of links: K= 49
1-theta-number of links: K= 86
2-theta-number of links: K= 154
1-alpha1-number of links: K= 45
2-alpha1-number of links: K= 37
1-alpha2-number of links: K= 26
2-alpha2-number of links: K= 29
1-beta1-number of links: K= 23
2-beta1-number of links: K= 24
1-beta2-number of links: K= 12
2-beta2-number of links: K= 19
1-beta3-number of links: K= 490
2-beta3-number of links: K= 489
1-gamma-number of links: K= 10
2-gamma-number of links: K= 67


### 5. Asymmetry in  each band

In [ ]:
left_region = ['Fp1', 'AF3', 'PO3','O1', 'CP1', 'FC1', 'P3', 'C3', 'F3', 'F7', 'FC5', 'CP5', 'T7', 'P7']
right_region = ['P8', 'T8', 'CP6', 'FC6', 'F8', 'F4', 'C4', 'P4', 'AF4', 'Fp2', 'FC2', 'CP2', 'O2', 'PO4']

for i in mapping:
    lr = 0 
    rr = 0
    lrc = 0
    rrc = 0
    for row in range(n_channels):
        for col in range(row+1):
            if binarized_matrix[f'{GROUP1}-{mapping[i]}'][row][col] == 1:
                if raw_1.info['ch_names'][row] in left_region or raw_1.info['ch_names'][col] in left_region:
                    lr += 1
                if raw_1.info['ch_names'][row] in right_region or raw_1.info['ch_names'][col] in right_region:
                    rr += 1
                # print(raw_1.info['ch_names'][row], raw_1.info['ch_names'][col])

            if binarized_matrix[f'{GROUP2}-{mapping[i]}'][row][col] == 1:
                if raw_1.info['ch_names'][row] in left_region or raw_1.info['ch_names'][col] in left_region:
                    lrc += 1
                if raw_1.info['ch_names'][row] in right_region or raw_1.info['ch_names'][col] in right_region:
                    rrc += 1
                # print(raw_1.info['ch_names'][row], raw_1.info['ch_names'][col])

    print(f"{GROUP1}: ",mapping[i], "left region: ", lr, "right region: ", rr, "proportion: ", lr/rr)
    print(f"{GROUP2}: ", mapping[i], "left region: ", lrc, "right region: ", rrc, "proportion: ", lrc/rrc)

1:  delta left region:  12 right region:  18 proportion:  0.6666666666666666
2:  delta left region:  34 right region:  24 proportion:  1.4166666666666667
1:  theta left region:  51 right region:  60 proportion:  0.85
2:  theta left region:  95 right region:  101 proportion:  0.9405940594059405
1:  alpha1 left region:  27 right region:  37 proportion:  0.7297297297297297
2:  alpha1 left region:  26 right region:  20 proportion:  1.3
1:  alpha2 left region:  16 right region:  22 proportion:  0.7272727272727273
2:  alpha2 left region:  22 right region:  17 proportion:  1.2941176470588236
1:  beta1 left region:  15 right region:  17 proportion:  0.8823529411764706
2:  beta1 left region:  20 right region:  10 proportion:  2.0
1:  beta2 left region:  8 right region:  8 proportion:  1.0
2:  beta2 left region:  16 right region:  9 proportion:  1.7777777777777777
1:  beta3 left region:  340 right region:  337 proportion:  1.0089020771513353
2:  beta3 left region:  336 right region:  338 proport

### 6. Difference matrix

In [ ]:
def return_index(channels_list, raw):
    indexes = []
    for channel in channels_list:
        for i in range(len(raw.info['ch_names'])):
            if raw.info['ch_names'][i]==channel:
                indexes.append(i)
    return(indexes)

In [ ]:
central_channels = return_index(['Pz', 'Oz', 'Fz', 'Cz'], raw_1)
central_channels
raw_1.info['bads'] = []

In [ ]:
difference_matrix = {}
for i in mapping:
    difference_matrix[mapping[i]] = binarized_matrix[f'{GROUP2}-{mapping[i]}'].astype(int) - binarized_matrix[f'{GROUP1}-{mapping[i]}'].astype(int) 
    # difference_matrix[mapping[i]] = binarized_matrix[f'{GROUP1}-{mapping[i]}'].astype(int) - binarized_matrix[f'{GROUP2}-{mapping[i]}'].astype(int)
    for k in range(n_channels):
        for l in range(n_channels):
            if difference_matrix[mapping[i]][k][l] == -1 or k in central_channels or l in central_channels:
                difference_matrix[mapping[i]][k][l] = 0

    # Yellow - Higher synchronisation
    # plt.imshow(difference_matrix[mapping[i]], interpolation='none')
    # plt.title(f'Functional connectivity - {[mapping[i]]} - Visualization')
    # plt.xticks([_ for _ in range(n_channels)], raw_1.info['ch_names'], rotation='vertical')
    # plt.yticks([_ for _ in range(n_channels)], raw_1.info['ch_names'], rotation='horizontal')
    # plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    # plt.show()

    # plot_sensors_connectivity(
    #     raw_1.info,
    #     difference_matrix[mapping[i]],
    #     cbar_label=f'{mapping[i]}-Connectivity',
    #     )

In [52]:
selected_channels = {}
for i in mapping:
    selected_channels[mapping[i]] = []
    for ch in range(32):
        # TODO: instead of number of edges wise, we can explore other algorithm
        if len(nx.from_numpy_array(difference_matrix[mapping[i]]).edges(ch)) > 6:
            selected_channels[mapping[i]].append(raw_1.info['ch_names'][ch])
print(selected_channels)

{'delta': ['CP2', 'CP1'], 'theta': ['P8', 'FC6', 'C4', 'FC2', 'CP2', 'PO3', 'O1', 'CP1', 'CP5', 'T7'], 'alpha1': [], 'alpha2': [], 'beta1': [], 'beta2': [], 'beta3': [], 'gamma': ['P8', 'PO3', 'CP1', 'C3', 'CP5', 'T7']}


Create dictionary of Regions in brain vs channels

In [53]:
regionvschannel = {
    'LC': ['C3', 'CP1', 'CP5', 'FC1', 'FC5'], # Left central 
    'LF': ['FP1', 'AF3', 'F3', 'F7'], # Left frontal
    'LT': ['T7'], # Left temporal
    'LPO': ['PO3',  'P3', 'P7', 'O1'], # Left parietal-occipital
    'RC': ['CP6', 'FC6', 'C4', 'FC2', 'CP2',], # Right central 
    'RF': ['F8', 'F4', 'AF4', 'FP2'], # Right frontal
    'RT': ['T8'], # Right temporal
    'RPO': ['P8', 'P4', 'PO4', 'O2'] , # Right parietal-occipital
}

In [58]:
# reverse mapping
channel_to_region = {}
for region, region_channels in regionvschannel.items():
    for channel in region_channels:
        channel_to_region[channel] = region

selected_regions = {}
for band, channels in selected_channels.items():
    regions = set()
    for channel in channels:
        if channel in channel_to_region:
            regions.add(channel_to_region[channel])
    selected_regions[band] = list(regions)


In [59]:
selected_regions

{'delta': ['RC', 'LC'],
 'theta': ['LPO', 'LT', 'RC', 'LC', 'RPO'],
 'alpha1': [],
 'alpha2': [],
 'beta1': [],
 'beta2': [],
 'beta3': [],
 'gamma': ['RPO', 'LPO', 'LC', 'LT']}